## 1. Short-Time Fourier Transform (STFT)

### How it functions
- **Cut into windows:** slice the EEG into short, overlapping chunks.
- **Analyze each chunk:** for each slice, check which frequencies are present and how strong they are.
- **Slide across time:** repeat for the next slice to build a movie of spectral content.
- **Trade-off:** short slices = sharp timing but fuzzy frequency; long slices = sharp frequency but blurrier timing.

### What you will see
- A **spectrogram** (time on x-axis, frequency on y-axis, color = power).
- In mental arithmetic: **alpha suppression (ERD)** over parietal areas and **theta increases** over frontal areas.

### Practical knobs
- **Window length:** typically 0.5–1.0 s for theta/alpha.
- **Overlap:** 50–90% for smoothness.
- **Window shape:** Hann/Hamming or multitaper to reduce leakage.

### Strengths / Pitfalls
- **Strengths:** simple, robust, reproducible; great for band-power and ERSP.
- **Pitfalls:** wrong window choice can blur features or miss rapid changes; edges may distort results.

## 2. Continuous Wavelet Transform (CWT)

### How it functions
- **Zoom-lens idea:** wavelets auto-adjust—short for high frequencies, long for low.
- **Pattern matching:** slide wave-shaped templates across the signal at different scales.

### What you will see
- A **scalogram** (time vs. frequency power) that often looks smoother than STFT.
- Clear detection of **transient bursts** (e.g., frontal theta bursts).

### Practical knobs
- **Mother wavelet:** complex Morlet (standard for EEG).
- **Frequency sampling:** ~2–45 Hz, with more detail at low frequencies.
- **Cone of influence (COI):** edges are unreliable—ignore them.

### Strengths / Pitfalls
- **Strengths:** adapts naturally to EEG; excellent for phase metrics (ITPC).
- **Pitfalls:** sensitive to parameter settings; must respect the COI.

## 3. Empirical Mode Decomposition (EMD) & Intrinsic Mode Functions (IMFs)

### How it functions
- **Peeling an onion:** repeatedly extract oscillatory layers from fastest to slowest.
- **Data-driven:** no pre-defined basis; the signal defines its own oscillations.
- **Result:** a few IMFs, each representing a rhythm that can drift in frequency.

### Why this is special
- EEG rhythms wander (e.g., alpha shifting from 10.5 Hz to 9.7 Hz). EMD follows these drifts directly.

### Variants
- **EEMD / CEEMDAN:** add tiny noise and average results to reduce mode mixing and improve stability.

### What you will see
- A stack of IMF signals: one looks “alpha-like,” another “theta-like,” each possibly drifting in frequency.

### Strengths / Pitfalls
- **Strengths:** captures amplitude and frequency modulation (AM–FM).
- **Pitfalls:** end effects; possible mode mixing; IMFs vary by subject/session.


## 4. Hilbert Transform & Hilbert--Huang Transform (HHT)

*(Typically applied after obtaining IMFs via EMD)*

### How it functions
- **Wrap IMFs in an envelope:** Hilbert transform gives instantaneous amplitude and phase.
- **Instantaneous frequency:** derived from the changing phase over time.
- **HHT:** combine results across IMFs to form a full time–frequency picture that follows rhythms moment-to-moment.

### What you will see
- **Amplitude traces:** strength of oscillations (e.g., alpha, theta) over time.
- **Instantaneous frequency traces:** whether rhythms drift up or down in frequency.
- **Hilbert spectrum:** a TF map built from these dynamic quantities.

### Strengths / Pitfalls
- **Strengths:** unmatched for tracking frequency drift and detailed amplitude changes.
- **Pitfalls:** unreliable when amplitude is very low; edge artifacts remain.


# Time--Frequency Analysis Methods for EEG

## 1. Short-Time Fourier Transform (STFT)

The STFT of a signal $\(x(t)\)$ with window $\(w(t)\)$, time shift $\(\tau\)$, and frequency $\(f\)$ is:

$\[
\text{STFT}_x(\tau,f) = \int_{-\infty}^{\infty} x(t)\, w(t-\tau)\, e^{-j 2 \pi f t}\, dt
\]$

The **spectrogram** is:

$\[
S(\tau,f) = \big|\text{STFT}_x(\tau,f)\big|^2
\]$

- Trade-off: shorter windows better time resolution, worse frequency resolution; longer windows better frequency resolution, worse time resolution.
- Key parameters: window length, overlap, window shape (Hann, Hamming).


## 2. Continuous Wavelet Transform (CWT)

For a mother wavelet $\(\psi(t)\)$, scale $\(s\)$, and translation $\(b\)$:

$\[
W_x(s,b) = \int_{-\infty}^{\infty} x(t)\, \frac{1}{\sqrt{s}} \, \psi^*\!\left(\frac{t-b}{s}\right) dt
\]$

The **scalogram** is:

$\[
\text{Scalogram}(s,b) = |W_x(s,b)|^2
\]$

Relationship between scale and pseudo-frequency:

$\[
f = \frac{f_c}{s \, \Delta t}
\]$

where $\(f_c\)$ is the wavelet center frequency.


## 3. Empirical Mode Decomposition (EMD)

A signal $\(x(t)\)$ is decomposed into Intrinsic Mode Functions (IMFs):

$\[
x(t) = \sum_{k=1}^{K} \text{IMF}_k(t) + r(t)
\]$

where $\(r(t)\)$ is the residual.  
Properties of an IMF:
1. Zero-crossings and extrema differ at most by one.  
2. Local mean is approximately zero.

Variants: EEMD, CEEMDAN (reduce mode mixing).


## 4. Hilbert Transform and Hilbert--Huang Transform (HHT)

The Hilbert transform of $\(x(t)\)$:

$\[
\mathcal{H}\{x\}(t) = \frac{1}{\pi} \, \text{p.v.} \int_{-\infty}^{\infty} \frac{x(\tau)}{t-\tau}\, d\tau
\]$

The analytic signal:

$\[
z(t) = x(t) + j\, \mathcal{H}\{x\}(t) = a(t) e^{j \phi(t)}
\]$

- Instantaneous amplitude: $\(a(t) = |z(t)|\)$  
- Instantaneous phase: $\(\phi(t) = \arg z(t)\)$  
- Instantaneous frequency:

$\[
f(t) = \frac{1}{2\pi} \frac{d\phi(t)}{dt}
\]$

**Hilbert Spectrum**:

$\[
H(f,t) = \sum_{k} a_k^2(t)\, \delta\!\big(f - f_k(t)\big)
\]$

**Marginal Spectrum**:

$\[
h(f) = \int H(f,t)\, dt
\]$


## 5. Welch's Method (Power Spectral Density Estimate)

Divide the signal into overlapping windows, compute the periodogram in each, and average:

$\[
\hat{P}_{xx}(f) = \frac{1}{K} \sum_{k=1}^{K} \frac{1}{U} \left| \sum_{n=0}^{L-1} x_k(n) w(n) e^{-j 2 \pi f n} \right|^2
\]$

where:
- $\(x_k(n)\)$: windowed segment,  
- $\(w(n)\)$: window function,  
- $\(U\)$: normalization constant,  
- $\(L\)$: window length,  
- $\(K\)$: number of segments.
